<a href="https://colab.research.google.com/github/Davioliveira1305/Simulacao-Estocastica/blob/main/Lista4_Simula%C3%A7%C3%A3o_Estoc%C3%A1stica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%pip install simpy

# 1ª Questão

In [29]:
import numpy as np
class Reparo:
  def __init__(self, env):
    self.env = env
    self.fila_formacao = simpy.Resource(env, capacity = 1) # Fila de formação eletrolítica
    self.fila_revestimento = simpy.Resource(env, capacity = 1) # Fila de revestimento
    self.fila_reparo = simpy.PriorityResource(env, capacity = 1) # Fila de reparo
    self.tam_fila_reparo = 0 # Tamanho da fila de reparos manuais
    self.tam_fila_formacao = 0
    self.tam_fila_revestimento = 0
    self.pedidos_no_sistema = 0

  def carga_de_trabalho(self, tam_fila, pedidos_no_sistema):
    return (tam_fila * 100) / pedidos_no_sistema

  def produzir_formacao(self, pedido):
    self.pedidos_no_sistema = self.pedidos_no_sistema + 1
    self.tam_fila_formacao = self.tam_fila_formacao + 1
    print(f'Pedido {pedido} chegou na fila de formação em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}, Carga de Trabalho(formação) = {self.carga_de_trabalho(self.tam_fila_formacao, self.pedidos_no_sistema)} %')
    with self.fila_formacao.request() as req:
      yield req
      yield self.env.timeout(np.random.uniform(8, 12))
      self.tam_fila_formacao = self.tam_fila_formacao - 1
      if np.random.rand() < 0.15:
        print(f'Pedido {pedido} precisou de reparo em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}')
        self.env.process(self.reparo(pedido,'formacao'))
      else:
        print(f'Pedido {pedido} foi para o processo de revestimento em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}')
        self.env.process(self.revestimento(pedido))

  def revestimento(self, pedido):
    self.tam_fila_revestimento = self.tam_fila_revestimento + 1
    print(f'Pedido {pedido} chegou na fila de revestimento em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}, Carga de Trabalho(revestimento) = {self.carga_de_trabalho(self.tam_fila_revestimento, self.pedidos_no_sistema)}%')
    with self.fila_revestimento.request() as req:
      yield req
      yield self.env.timeout(np.random.uniform(4, 8))
      self.tam_fila_revestimento = self.tam_fila_revestimento - 1
      if np.random.rand() < 0.12:
        print(f'Pedido {pedido} precisou de reparo em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}')
        self.env.process(self.reparo(pedido,'revestimento'))

      else:
        self.pedidos_no_sistema = self.pedidos_no_sistema - 1
        print(f'Pedido {pedido} ficou pronto em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}')

  def reparo(self, pedido, tipo):
    self.tam_fila_reparo = self.tam_fila_reparo + 1
    print(f'Pedido {pedido} chegou no reparo em {self.env.now}, Tamanho da fila de reparo = {self.tam_fila_reparo}, Carga de Trabalho(reparo) = {self.carga_de_trabalho(self.tam_fila_reparo, self.pedidos_no_sistema)}%')
    if tipo == 'formacao': prio = 0 # Processo de formação tem prioridade na fila de reparos
    elif tipo == 'revestimento': prio = 1
    with self.fila_reparo.request(priority = prio) as req:
      yield req
      if tipo == 'formacao':
        yield self.env.timeout(np.random.uniform(10, 20))
        self.tam_fila_reparo = self.tam_fila_reparo - 1
        print(f'Pedido {pedido} voltou para o processo de formação em {self.env.now}, Tamanho da fila de reparo = {self.tam_fila_reparo}')
        self.env.process(self.produzir_formacao(pedido))
      else:
        yield self.env.timeout(np.random.uniform(15, 24))
        self.tam_fila_reparo = self.tam_fila_reparo - 1
        print(f'Pedido {pedido} voltou para o processo de revestimento em {self.env.now}, Tamanho da fila de reparo = {self.tam_fila_reparo}')
        self.env.process(self.revestimento(pedido))

def gera_pedidos(env, reparo):
  pedido = 0
  while True:
    yield env.timeout(np.random.triangular(3,7,14))
    pedido = pedido + 1
    env.process(reparo.produzir_formacao(pedido))

env = simpy.Environment()
fila_reparo = Reparo(env)

env.process(gera_pedidos(env, fila_reparo))

env.run(until=365*24)



Pedido 1 chegou na fila de formação em 6.511723063183375, Tamanho da fila de formação = 1, Carga de Trabalho(formação) = 100.0 %
Pedido 2 chegou na fila de formação em 13.764166727765856, Tamanho da fila de formação = 2, Carga de Trabalho(formação) = 100.0 %
Pedido 1 precisou de reparo em 14.99505587866043, Tamanho da fila de formação = 1
Pedido 1 chegou no reparo em 14.99505587866043, Tamanho da fila de reparo = 1, Carga de Trabalho(reparo) = 50.0%
Pedido 3 chegou na fila de formação em 20.969162474215967, Tamanho da fila de formação = 2, Carga de Trabalho(formação) = 66.66666666666667 %
Pedido 2 foi para o processo de revestimento em 25.631750417432755, Tamanho da fila de formação = 1
Pedido 2 chegou na fila de revestimento em 25.631750417432755, Tamanho da fila de revestimento = 1, Carga de Trabalho(revestimento) = 33.333333333333336%
Pedido 1 voltou para o processo de formação em 26.04312484706635, Tamanho da fila de reparo = 0
Pedido 1 chegou na fila de formação em 26.043124847066